In [1]:
# file dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle
import argparse
from os import listdir

In [2]:
# data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# directory of all jobs on given date
source_dir = '/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/'

# get list of files in directory
dates_list = [ date for date in listdir(source_dir) ]

In [4]:
# access and open all valid jobs in range
# append to list jobs
valid_jobs = []

for date in dates_list:
    
    # do not open empty folders
    if len( listdir(source_dir+date) ) != 0 and len(valid_jobs) < 500:
        for job in listdir(source_dir+date):
            
            # open job file if possible
            try:
                pickle_file = open( source_dir+date+'/'+job, 'rb')
                jobid = pickle.load(pickle_file)
                print jobid
                
                # only save jobs that ran longer than 1 hour
                if len(jobid.times) > 4:
                    valid_jobs.append(jobid)
                    print "added"
                    
                pickle_file.close()
            except:
                next
    else:
        next

added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


...

added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added


In [5]:
valid_jobs

In [6]:
len(valid_jobs)

746

### Format and Save Each Job
- format all valid jobs to dataframe
- save as csv to export to upgraded project

In [7]:
# select and rename stats
def formatTitles ( df ):
    return df.rename( columns={"index": "Job Name",
              "amd64_pmc": "AMD Opteron performance counters (per core)",
              "intel_hsw": "Intel Haswell Processor (HSW) (per core)",
              "intel_hsw_ht": "Intel Haswell Processor - Hyper-threaded (per logical core)",
              "intel_nhm": "Intel Nehalem Processor (NHM) (per core)",
              "intel_uncore": "Westmere Uncore (WTM) (per socket)",
              "intel_snb": "Intel Sandy Brige (SNB) or Ivy Bridge (IVB) Processor (per core)",
              "intel_snb(hsw)_cbo": "Caching Agent (CBo) for SNB (HSW) (per socket)",
              "intel_snb(hsw)_pcu": "Power Control Unit for SNB (HSW) (per socket)",
              "intel_snb(hsw)_imc": "Integrated Memory Controller for SNB (HSW) (per socket)",
              "intel_snb(hsw)_qpi": "QPI Link Layer for SNB (HSW) (per socket)",
              "intel_snb(hsw)_hau": "Home Agent Unit for SNB (HSW) (per socket)",
              "intel_snb(hsw)_r2pci": "Ring to PCIe Agent for SNB (HSW) (per socket)",
              "ib": "Infiniband usage (default)",
              "ib_sw": "InfiniBand usage (sw)",
              "ib_ext": "Infiniband usage (ext)",
              "llite": "Lustre filesystem usage (per mount)",
              "lnet": "Lustre network usage (lnet)",
              "mdc": "Lustre network usage (mdc)",
              "mic": "MIC scheduler account (per hardware thread)",
              "osc": "Lustre filesystem usage (osc)",
              "block": "Block device statistics (per device)",
              "cpu": "Scheduler accounting (per CPU)",
              "mem": "Memory usage (per socket)",
              "net": "Network device usage (per device)",
              "nfs": "NFS system usage",
              "numa": "NUMA statistics (per socket)",
              "proc": "Process specific data (MaxRSS, executable name etc.)",
              "ps": "Process statistics",
              "sysv_shm": "SysV shared memory segment usage",
              "tmpfs": "Ram-backed filesystem usage (per mount)",
              "vfs": "Dentry/file/inode cache usage",
              "vm": "Virtual memory statistics"
                            })

In [ ]:
def setByTimes ():
    all_stats = { time:{ stat:[] for stat in type_avgs } for time in times }
    
    for type_avg,data in type_avgs.iteritems():
        for dev in range( len(data) ):
            all_stats[dt.time(21, 59, 44)][type_avg] .append(  data[dev][0]  .round())
            all_stats[dt.time(22, 9, 45)] [type_avg] .append(   data[dev][1] .round())   
            all_stats[dt.time(22, 19, 45)][type_avg] .append(  data[dev][2]  .round()) 
            all_stats[dt.time(22, 29, 45)][type_avg] .append(  data[dev][3]  .round())
            all_stats[dt.time(22, 39, 45)][type_avg] .append(  data[dev][4]  .round())
            all_stats[dt.time(22, 49, 45)][type_avg] .append(  data[dev][5]  .round())   
            all_stats[dt.time(0, 38, 3)]  [type_avg] .append(    data[dev][6].round())

In [ ]:
def descriptive_stats(  )
    # compile and save averages
    dev_stats = { type_stat:{} for type_stat in stat_types}
    total = 0
    
    for time,stat in all_stats.iteritems():
        for stat,avgList in stat.iteritems():
            dev_stats[stat]["Average by Device"] = avgList
            dev_stats[stat]["Overall Average"] = np.mean(avgList)
            dev_stats[stat]["Standard Deviation"] = np.std(avgList)
            dev_stats[stat]["High"] = np.max(avgList)
            dev_stats[stat]["Low"] = np.min(avgList)
            
            all_stats[time][stat] = np.mean(avgList)
    total += 1
            
    desc_df = pd.DataFrame( dev_stats ).drop('Average by Device', axis=0 )
    df_time.to_csv(path_or_buf=f"./descriptive_stats/job_{total}.csv")

In [ ]:
def formatJob ( jobid ):

    # general stats
    start = pd.to_datetime(round(jobid.start_time), unit='s').time()
    end = pd.to_datetime(round(jobid.end_time), unit='s').time()
    numCycles = len(jobid.times) + 2
    stat_types = jobid.hosts['comet-03-09'].stats.keys()
    
    type_avgs = {}
    times = [start]
    devices = []
    
    for host_name, host in jobid.hosts.iteritems():
        typ='all'
        dev='all'
        
        if 'all' in typ: typ_keys = host.stats.keys()
        else: typ_keys = typ
            
        for time in host.times:
            times.append( pd.to_datetime(round(time), unit='s').time() )
        
        for type_name in typ_keys:
            type_data = host.stats[type_name]
            type_avgs[type_name] = []
            
            if 'all' in dev: dev_keys = type_data.keys()
            else: dev_keys = dev
                
            for dev_name in dev_keys:          
                devices.append( (dev_name,type_name) )
                type_avgs[type_name].append( [np.mean(row).round(decimals=3)
                                       for row in type_data[dev_name]] )
    times.append(end)

In [ ]:
# index by cycle
# define indices
indexList = ["First", "Second", "Third", "Fourth", "Fifth", "Sixth", "Seventh"]

# index by time of check
df_1 = pd.DataFrame( all_stats[dt.time(21, 59, 44)], index=["First"] )#.set_index(indexList)
df_2 = pd.DataFrame( all_stats[dt.time(22, 9, 45)],  index=["Second"] ) #.set_index(indexList)
df_3 = pd.DataFrame( all_stats[dt.time(22, 19, 45)], index=["Third"] )#.set_index(indexList)
df_4 = pd.DataFrame( all_stats[dt.time(22, 29, 45)], index=["Fourth"] )#.set_index(indexList)
df_5 = pd.DataFrame( all_stats[dt.time(22, 39, 45)], index=["Fifth"] )#.set_index(indexList)
df_6 = pd.DataFrame( all_stats[dt.time(22, 49, 45)], index=["Sixth"] )#.set_index(indexList)
df_7 = pd.DataFrame( all_stats[dt.time(0, 38, 3)],   index=["Seventh"] )  #.set_index(indexList)

# merge dataframes
df_time = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7])
df_time.index.name = "Cycle"
df_time

In [ ]:
for i in range( len(valid_jobs) ):
    formatJob( valid_jobs[i] )
    